#### Test Pipeline
This is a Julia Notebook to check the correctness of the C++ code. 

The notebook proceeds as follows: 
- Generate a synthetic instance
- Run the original implementation of the algorithm (pure Julia)
- Run the C++ implementation of the algorithm

#### Step 1: Data Generation

Generates a ground truth covariance matrix S of the form $I + \beta x_1 x_1^\top + \beta x_2 x_2^\top$, where 
- $x_1, x_2$ are $k$-sparse, with non-overlapping support 
- $\beta$ controls the signal-to-noise ratio

Then, samples $n$ multivariate normal observation from $\mathcal{N}(0,S)$ and constructs the empirical covariance matrix $\Sigma$

In [1]:
using Random, LinearAlgebra
p = 10 #Dimension
r = 2 #Number of sparse PCs
k = 4 #Sparsity of each PC
β = 1 #Signal strength


x1 = zeros(p); x1[1:k] = sign.(rand(k) .- .5)
x2 = zeros(p); x2[(k+1):(k+k)] = sign.(rand(k) .- .5)

# x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap)] = sign.(rand(k_overlap) .- .5)
# x2[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)] = -x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)]
# x2[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)] = x1[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)]

@assert sum(abs.(x1) .> 0) == k
@assert sum(abs.(x2) .> 0) == k
@assert abs(dot(x1,x2)) ≤ 1e-10

shufflecoords = randperm(p)
x1 = x1[shufflecoords]; x2=x2[shufflecoords] 

x1 /= norm(x1); x2 /= norm(x2) 

S = β*x1*x1'+β*x2*x2'+ Matrix(1.0*I, p, p)
S = (S + S')/2

10×10 Matrix{Float64}:
  1.25   0.25   0.0    0.0    0.0    0.0   -0.25   0.25  0.0  0.0
  0.25   1.25   0.0    0.0    0.0    0.0   -0.25   0.25  0.0  0.0
  0.0    0.0    1.25  -0.25   0.25  -0.25   0.0    0.0   0.0  0.0
  0.0    0.0   -0.25   1.25  -0.25   0.25   0.0    0.0   0.0  0.0
  0.0    0.0    0.25  -0.25   1.25  -0.25   0.0    0.0   0.0  0.0
  0.0    0.0   -0.25   0.25  -0.25   1.25   0.0    0.0   0.0  0.0
 -0.25  -0.25   0.0    0.0    0.0    0.0    1.25  -0.25  0.0  0.0
  0.25   0.25   0.0    0.0    0.0    0.0   -0.25   1.25  0.0  0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   1.0  0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0  1.0

In [2]:
n = 1000 #Sample size

using Distributions
d = MvNormal(zeros(p), S)
X = rand(d, n) #p by N matrix of observations

Sn = cov(X') #Sample covariance matrix

10×10 Matrix{Float64}:
  1.3418       0.184349     -0.0423763   …  -0.0212752   -0.00956836
  0.184349     1.24818      -0.0113497       0.0107353   -0.0442101
 -0.0423763   -0.0113497     1.18754        -0.0132997   -0.00410034
  0.0228249   -0.0315741    -0.306527        0.044302     0.0193702
 -0.0531532   -0.000604907   0.25362         0.00436462   0.0546951
  0.0193823    0.0197881    -0.270447    …   0.0169479   -0.027697
 -0.268579    -0.230394      0.00102432      0.0178985    0.0882726
  0.232942     0.26299      -0.0426173       0.0731692   -0.0503366
 -0.0212752    0.0107353    -0.0132997       0.962552    -0.0758453
 -0.00956836  -0.0442101    -0.00410034     -0.0758453    0.911611

In [3]:
show(stdout, "text/plain", Sn)

10×10 Matrix{Float64}:
  1.3418       0.184349     -0.0423763    0.0228249  -0.0531532     0.0193823   -0.268579     0.232942   -0.0212752   -0.00956836
  0.184349     1.24818      -0.0113497   -0.0315741  -0.000604907   0.0197881   -0.230394     0.26299     0.0107353   -0.0442101
 -0.0423763   -0.0113497     1.18754     -0.306527    0.25362      -0.270447     0.00102432  -0.0426173  -0.0132997   -0.00410034
  0.0228249   -0.0315741    -0.306527     1.22263    -0.29982       0.25559     -0.014924     0.0192916   0.044302     0.0193702
 -0.0531532   -0.000604907   0.25362     -0.29982     1.26994      -0.25687     -0.0307962    0.0141107   0.00436462   0.0546951
  0.0193823    0.0197881    -0.270447     0.25559    -0.25687       1.23601     -0.00447638  -0.0177363   0.0169479   -0.027697
 -0.268579    -0.230394      0.00102432  -0.014924   -0.0307962    -0.00447638   1.22703     -0.240063    0.0178985    0.0882726
  0.232942     0.26299      -0.0426173    0.0192916   0.0141107    -0.017

In [4]:
[x1 x2]

10×2 Matrix{Float64}:
 -0.5   0.0
 -0.5   0.0
  0.0   0.5
  0.0  -0.5
  0.0   0.5
  0.0  -0.5
  0.5   0.0
 -0.5   0.0
  0.0   0.0
  0.0   0.0

In [5]:
[k, k]

2-element Vector{Int64}:
 4
 4

#### Step 2: Julia benchmark

Applies the Julia code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [6]:
include("algorithm2.jl")

findmultPCs_deflation (generic function with 1 method)

In [7]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(S, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 


          1 |                0.333 |                  1.00e-07 |      1.856 


10×2 Matrix{Float64}:
  0.0  -0.5
  0.0  -0.5
 -0.5   0.0
  0.5   0.0
 -0.5   0.0
  0.5   0.0
  0.0   0.5
  0.0  -0.5
  0.0   0.0
  0.0   0.0

In [8]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Sn, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 
          1 |                0.348 |                  2.00e+00 |      0.037 


          2 |                0.348 |                  2.00e+00 |      0.063 
          3 |                0.348 |                  2.00e+00 |      0.102 
          4 |                0.348 |                  2.00e+00 |      0.128 
          5 |                0.340 |                  1.00e-07 |      0.164 


          6 |                0.340 |                  1.00e-07 |      0.198 
          7 |                0.340 |                  1.00e-07 |      0.224 
          8 |                0.340 |                  1.00e-07 |      0.258 


10×2 Matrix{Float64}:
 -0.532853   0.0
 -0.477478   0.0
  0.0        0.484719
  0.0       -0.514639
  0.0        0.516055
  0.0       -0.483613
  0.504861   0.0
 -0.482906   0.0
  0.0        0.0
  0.0        0.0

In [ ]:
show(stdout, "text/plain", x_best)

#### Step 3: R/C++ implementation

Applies the R code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [9]:
using RCall

In [10]:
R"""library(sPCAmPC)"""
# R"""
# library(devtools)
# reload(pkg = "sPCAmPC", quiet = FALSE)"""


RObject{StrSxp}
[1] "sPCAmPC"   "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     


In [11]:
R"""

TestMat <- $S 

cpp_findmultPCs_deflation(TestMat, 2, c(4, 4), numIters=20)
"""

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern:  4 4

  Iteration |      Objective value |   Orthogonality Violation |       Time
          1 |                0.333 |                  2.00e+00 |      0.023
          2 |                0.333 |                  1.00e-07 |      0.047


RObject{RealSxp}
      [,1] [,2]
 [1,]  0.0  0.5
 [2,]  0.0  0.5
 [3,]  0.5  0.0
 [4,] -0.5  0.0
 [5,]  0.5  0.0
 [6,] -0.5  0.0
 [7,]  0.0 -0.5
 [8,]  0.0  0.5
 [9,]  0.0  0.0
[10,]  0.0  0.0


In [12]:
R"""

TestMat <- $Sn 

cpp_findmultPCs_deflation(TestMat, 2, c(4, 4), numIters=20)
"""

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern:  4 4

  Iteration |      Objective value |   Orthogonality Violation |       Time
          1 |                0.348 |                  2.00e+00 |      0.024
          2 |                0.348 |                  2.00e+00 |      0.050
          3 |                0.348 |                  2.00e+00 |      0.073
          4 |                0.348 |                  2.00e+00 |      0.096
          5 |                 0.34 |                  1.00e-07 |      0.119
          6 |                 0.34 |                  1.00e-07 |      0.143
          7 |                 0.34 |                  1.00e-07 |      0.166


          8 |                 0.34 |                  1.00e-07 |      0.190


RObject{RealSxp}
            [,1]       [,2]
 [1,]  0.5383958  0.0000000
 [2,]  0.4770751  0.0000000
 [3,]  0.0000000 -0.4853794
 [4,]  0.0000000  0.5149751
 [5,]  0.0000000 -0.5143314
 [6,]  0.0000000  0.4844282
 [7,] -0.5021696  0.0000000
 [8,]  0.4799531  0.0000000
 [9,]  0.0000000  0.0000000
[10,]  0.0000000  0.0000000


In [13]:
[x1 x2]

10×2 Matrix{Float64}:
 -0.5   0.0
  0.0  -0.5
  0.0   0.5
  0.0  -0.5
  0.0   0.0
 -0.5   0.0
  0.0  -0.5
  0.0   0.0
  0.5   0.0
 -0.5   0.0